# Diagnóstico de Doença Renal Crónica com Tensorflow

In [1]:
import numpy as np  
import tensorflow as tf
import tensorflow.contrib.tensorboard.plugins.projector as tf_projector
import scipy.io.arff
from time import time
import sklearn.model_selection
import random
import os

In [2]:
tf.__version__

'1.0.1'

## Introdução

Este tutorial cobre a utilização da biblioteca Tensorflow para o diagnóstico de Doença Renal Crónica com uma rede neuronal do tipo feed-forward. Este processo envolve os seguintes passos:

- Pré-processamento dos dados (Scipy e Numpy)
- Construção da rede (Tensorflow)
- Treino e avaliação da rede (Tensorflow)
- Visualização de métricas de treino (Tensorboard)
- Visualização do dataset (Tensorboard)

O dataset usado, assim como a descrição de cada atributo e classe está disponível em https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease

Tensorflow é uma biblioteca de computação numérica baseada em grafos de computação. Construimos um grafo adicionando-lhe nós que representam operações matemáticas. Estas operam sobre tensores, a estrutura de dados que dá o nome à biblioteca, que são a generalização de escalares, vectores, matrizes, etc. a qualquer número de dimensões. A vantagem da utilização deste grafo é que permite que toda a computação a ser realizada seja conhecida pelo ambiente de execução desde que este é iniciado. A utilização deste modelo traz várias vantagens: 

- Redução do overhead de comunicação entre o código Python e o Tensorflow, escrito em C++
- Variedade de otimizações das operações que seria impossível se estas tivessem de ser executadas uma a uma
- Cálculo de derivadas de expressões baseadas na regra da cadeia para funções compostas

Este último ponto é essencial para o usa da biblioteca para a implementação de redes neuronais. Para qualquer expressão matemática diferenciável, a biblioteca é capaz de automaticamente calcular o gradiente em ordem a cada variável, eliminando a necessidade de implementar o algoritmo de backpropagation.

In [3]:
# tf usado para aceder à biblioteca
one = tf.constant(1)                            # tensor contem escalar constante constante

# placeholder é um valor que será fornecido ao correr o grafo
x = tf.placeholder(                             
    dtype="int32",                              # tipo do placeholder
    shape=[],                                   # forma [] corresponde um escalar
    name="x"                                    # nome do tensor
)

# adiciona operação de soma entre x e one ao grafo
# apenas toma um valor ao correr o grafo com run()
c = tf.add(one, x)                              # equivalente a one + x

# sessões encapsulam ambiente e o estado em que correm o grafo, with encarrega-se de iniciar e fechar a sessão
with tf.Session() as sess:                     
    # na sessão sess, calcular o valor de c, dando o valor 2 a x e atribuir o resultado à variável res
    res = sess.run(c, feed_dict={x:2})
    print("c = {}".format(res))

c = 3


A avaliação apenas é feita ao chamar a função run(), e apenas nos nós que são necessários para calcular os valores pedidos.

In [4]:
a = tf.constant(1)
b = tf.constant(2)
c = tf.constant(3)
d = tf.constant(4)

ab = tf.add(a,b)
bc = tf.add(b,c)
cd = tf.add(c,d)

with tf.Session() as sess:
    a_vl, ab_val, bc_val = sess.run([a, ab, bc]) # apenas são avaliadas as operações de que dependem as operações 
                                                 # passadas em run(), neste caso, cd não é avaliado. 
                                                 # run() retorna um valor por cada operação que lhe é passada
    print("a  = {}\nab = {}\nbc = {}".format(a_vl, ab_val, bc_val))


a  = 1
ab = 3
bc = 5


## Pré-processamento de dados

O primeiro passo para o treino consiste no pré-processamento dos dados. O dataset contém 400 individuos, cada um com 24 
atributos e uma classificação binária, com ou sem doença. Alguns destes atributos são nominais e terão de ser convertidos 
para numéricos. Cada atributo nominal com n valores possíveis é substituído por n atributos binários, que indicam se o 
indíviduo tem ou não aquele valor no atributo original. Os atributos que já são binários não necessitam desta conversão 
mas esta é feita neste caso para simplificar o tratamento dos dados.

Inicialmente é feita a conversão dos campos nominais para vários binários, depois são substituídos os valores desconhecidos
e, finalmente, os dados são normalizados.

In [5]:
filename = "chronic_kidney_disease.arff"
data, meta = scipy.io.arff.loadarff(filename)
data_copy = np.copy(data)

"""
lb = skl.preprocessing.LabelBinarizer()
for name, ty in meta.__dict__.get("_attributes").items():
    print(name, ty)
    if ty[0] == "nominal":
        lb.fit([st.encode(encoding='UTF-8') for st in ty[1]])
        temp = lb.transform(data[name]).astype(np.float64)
        print(temp)
"""

from numpy.lib.recfunctions import append_fields
from numpy.lib.recfunctions import drop_fields

def pre_process(data, class_name):
    # contagem do número de classes existentes para serem separados dos atributos posteriormente
    num_classes = 0
    # vai permitir substituir os valores dum nparray por floats
    # se o valor for desconhecido, é substituído por NaN, se o valor
    # for igual ao passado, é substituido por 1, se não, é substituído por 0
    vec = lambda preset: np.vectorize(lambda v: np.nan if v == b'?' else 1. if v == preset else 0.)
    for i, field in enumerate(data.dtype.names):
        type_of_field = meta.types()[i]                       # tipo do campo pode ser nominal ou numérico
        
        if type_of_field == "nominal":                        # apenas serão tratados os campos nominais
            
            valid = data[field][data[field] != b'?']          # extrair os valores conhecidos
            
            unique = np.unique(valid).tolist()                # lista com os valores possíveis do atributo atual
            
            for val in unique:
                # para cada valor val possível do atributo field, criamos um novo campo chamado field_val
                # que terá o valor 1 para os indivíduos com o valor val no atributo field_val e 0 nos restantes
                new_field = field + "_" + val.decode("utf-8") 
                data = append_fields(
                    data,
                    new_field,
                    vec(val)(data[field])
                )
            data = drop_fields(data, field) # eliminar campo original
            
    for i, field in enumerate(data.dtype.names):
        if field[:len(class_name)] == class_name:             # não normalizar classes
            num_classes += 1
            continue
        valid = data[field][~np.isnan(data[field])]           # extrair valores conhecidos
        
        std = valid.std()                                     
        mean = valid.mean()
        
        data[field][np.isnan(data[field])] = mean             # subsituir valores desconhecidos pela média dos valores 
                                                              # conhecidos do campo atual
            
        data[field] = (data[field] - mean) / std              # normalizar valores, subtraindo a média e divindo pelo
                                                              # desvio padrão do campo atual, resulta numa distribuição 
                                                              # com média 0 e desvio padrão unitário
    return num_classes, data

num_classes, data = pre_process(data=data, class_name="class")

In [6]:
# extracção dos valores do structered array para ndarrays

# converter o structured array para um array numpy de floats
data_array = data.view(np.float64)

# mudar a forma do array para ter número de linhas correspondente ao número de indíviduos
data_array = data_array.reshape(data.shape[0], -1)

# extrair apenas os atributos
# para todas as linhas, extrair da primeira coluna até a última - num_classes coluna
inputs = data_array[:, :-num_classes]

# extrair as classes
# para todas as linhas, extrair as últimas num_classes colunas
labels = data_array[:, -num_classes:]

## Construção da rede

De modo a construir a rede, para cada camada da rede, adicionando as seguintes operações:
- multiplicação matricial entre output da camada anterior e pesos da camada actual
- soma dos biases da camada atual
- se camada não for final, função de ativação

$$o^{i} = o^{i-1} . W^{i} + b^{i} $$

A utilização de keyword arguments é opcional e é usada aqui para explicitar os argumentos usados.

In [7]:
# construção de uma camada da rede com  num_units neurónios
def make_layer(inputs, num_units, name, final = True):
    
    with tf.name_scope(name):                          # name scope permite agrupar um conjunto de operações
        
        # variáveis são usadas para os valores que vão sendo alterados, têm trainable=True por defeito
        # e são por isso atualizadas automaticamente de modo a diminuir o erro no processo de treino
        weights = tf.Variable(                         
            initial_value= tf.truncated_normal(        # pesos inicializados aleatóriamente com uma distribuição normal
                [inputs.shape[1].value, num_units],    # matriz de tamanho (nr colunas de input para a camada) * (nº de neurónios)
                dtype=tf.float64                       # tipo dos valores gerados
            ),                     
            name=name + "_weights"                     # nome permite identificar fácilmente a variável no Tensorboard
        )
        
        biases = tf.Variable(
            tf.zeros(                                  # inicializar a 0s
                [num_units],                           # vetor com um valor por neurónio
                dtype=tf.float64
            ),    
            name=name + "_biases"
        )
        
        #permite a visualização da distribuição das variáveis ao longo do treino
        tf.summary.histogram(
            name="weights", 
            values=weights
        ) 
        tf.summary.histogram(
            name="biases",
            values=biases
        )
        
        # multiplicação de matrizes
        ret = tf.matmul(inputs, weights) + biases      # soma com tensores equivalente a tf.add()
        
        if not final:
            # função de ativação ReLU https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
            # não é usada na camada final
            ret = tf.nn.relu(features=ret)
            
    return ret

def make_net(inputs, hidden_units, classifications):
    in_val = inputs
    for i, num_units in enumerate(hidden_units):
        # criar uma camada intermédia para cada elemento de hidden_units
        in_val = make_layer(
            in_val, 
            num_units, 
            "layer_{}".format(i),
            False
        )
    # camada final
    out_val = make_layer(
        in_val, 
        classifications.shape[1].value, 
        "final_layer",  
        True
    )
    
    with tf.name_scope("post"): # cálculo de probabilidades, custo, 
        # softmax transforma outputs da camada final nas probabilidades para cada classe
        # https://en.wikipedia.org/wiki/Softmax_function
        probabilities = tf.nn.softmax(
            logits=out_val, 
            name="soft_max"
        )
        
        cost = tf.reduce_mean(                           # custo é média da cross entropy para todas as classes
            tf.nn.softmax_cross_entropy_with_logits(     # http://neuralnetworksanddeeplearning.com/chap3.html
                labels=classifications,                  # classificações vindas dos dataset
                logits=out_val,                          # valores de saída da camada final
                name="cross"
            ), 
            name="cost"
        )
        
        # optimizer permite treinar os pesos e biases da rede
        # ao avaliar esta operação, o gradiente é calculado automáticamente
        # e propagado pela rede, atualizando o valor dos pesos e biases
        # existem vários otimizadores definidos em tf.train
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=0.01,                          # usamos uma learning rate constante, mas pode variar com o step
            name="opt"
        ).minimize(                                      # indicar que o custo deve ser minimizado
            cost, 
            name="minimizer"
        )           
        
        # a rede escolhe a classificação com um maior valor de output da camada final
        # classificação é o índice do maior valor
        predictions = tf.argmax(
            input=out_val,
            dimension=1,
            name="predictions"
        )
        
        # são correctas as previsões iguais às labels do dataset
        correct_predictions = tf.equal(
            x=predictions,
            y=tf.argmax(
                input=classifications,                   # vindos do data set
                dimension=1
            ), 
            name="correct"
        )
        
        # precisão é a fracção de previsões feitas corretamente
        accuracy = tf.reduce_mean(
            tf.cast(                                     # conversão de booleanos para floats para calcular a média
                x=correct_predictions, 
                dtype=tf.float64
            ), 
            name="acc"
        )
        
        # permite a visualização das métricas de custo e precisão ao longo do treino
        tf.summary.scalar(
            name="acc", 
            tensor=accuracy
        )
        tf.summary.scalar(
            name="cost", 
            tensor=cost
        )
    
    return probabilities, optimizer, predictions, correct_predictions, accuracy
        

## Treino e avaliação da rede

In [8]:
# fazer reset ao grafo, para eliminar nós já introduzidos
tf.reset_default_graph()

# vai conter os atributos dos invíduos
X = tf.placeholder(                                  
    dtype="float64",                                     # tipo de dados do tensor
    
    shape=[None, inputs.shape[1]],                       # o tensor passado será uma matriz com uma linha por indivíduo
                                                         # e uma coluna por atributo. O número de indíviduos pode variar
                                                         # e por isso é usado o valor None para este
    name="X"
)

# vai conter as classificações vindas do dataset
y = tf.placeholder(
    dtype="float64", 
    shape=[None, labels.shape[1]], 
    name="y"
)

keep_prob = tf.placeholder(
    dtype=tf.float64, 
    name="keep_prob"
)

# chamada da função de construção da rede definida acima
probabilities, optimizer, predictions, correct_predictions, accuracy = make_net(X, [4,4], y)

# operação de inicialização das variávies (pesos e biases)
init = tf.global_variables_initializer()

# número máximo de iterações
num_epochs = 1000

# percentagem dos indíviduos a usar no conjunto de teste
# é usado para verificar o poder de previsão da rede em indíviduos
# que não foram usados no treino
test_size = 0.5

# percentagem do indíviduos do conjunto de treino a usar no conjunto de validação
validation_size = 0.2

# operação que junta todos as operações de summary
merged = tf.summary.merge_all()

random.seed(42)

LOG_DIR = os.path.join('.', 'logs')

def make_file_writer(dataset, curr_time, session):
    return tf.summary.FileWriter(
        os.path.join(LOG_DIR, "{}", "{}").format(dataset, curr_time),
        session.graph
    )

# baralha os indivíduos e separa-os em conjunto de treino e teste
# stratified significa que a divisão é feita mantendo a mesma proporção
# de indivíduos de cada classe em cada 
kf = sklearn.model_selection.StratifiedShuffleSplit(
    n_splits=1, 
    test_size=test_size
)


# iterar sobre as diferentes formas de separar os dados em treino e teste
# como n_splits=1 acima, apenas é retornada uma
# os dados são separados uma única vez em treino e teste
# em cada iteração, os dados de treino vão ser separados em treino e validação
for train_temp_i, test_i in kf.split(inputs, labels):   
    
    # split retorna os indices a usar, aqui obtemos os dados que os indices referem
    train_temp_inputs = inputs[train_temp_i]
    train_temp_labels = labels[train_temp_i]
    test_inputs       = inputs[test_i]
    test_labels       = labels[test_i]
    
    with tf.Session() as sess:
        # usamos o tempo atual para por os dados de cada run num directório diferente
        curr_time = time()
        
        # para escrever estatísticas em ficheiros ao longo do processo de treino
        # escrevemos as estatísticas de treino, validação e teste em ficheiros diferentes para os comparar
        train_writer = make_file_writer("train", curr_time, sess)
        valid_writer = make_file_writer("valid", curr_time, sess)
        test_writer  = make_file_writer("test" , curr_time, sess)
        
        sess.run(init)
        for i in range(num_epochs):
            # em cada iteração, separar os dados de treino nos conjuntos de validação e de treino
            # mantendo proporção de indíviduos de cada classe
            kf2 = sklearn.model_selection.StratifiedShuffleSplit(
                n_splits=1, 
                test_size=validation_size
            )
            
            for train_i, validation_i in kf2.split(train_temp_inputs, train_temp_labels):
                # aceder aos indices retornados
                train_inputs      = train_temp_inputs[train_i]  
                train_labels      = train_temp_labels[train_i]
                validation_inputs = train_temp_inputs[validation_i]
                validation_labels = train_temp_labels[validation_i]
                
                # treino da rede
                _, stats = sess.run(
                    [optimizer, merged],
                    feed_dict={X: train_inputs, y: train_labels}
                )
                
                # escrever as estatísticas de treino para o passo i
                train_writer.add_summary(
                    summary=stats,
                    global_step=i
                )        
                
                # validação (como não é passada a operação de minimização, a rede não é treinada)
                acc, stats = sess.run(
                    [accuracy, merged], 
                    feed_dict={X: validation_inputs, y: validation_labels}
                )
                
                # escrever as estatísticas de validação para o passo i
                valid_writer.add_summary(
                    summary=stats,
                    global_step=i
                )        
            
            # periodicamente verificar a performance da rede no conjunto de treino
            # a rede nunca é treinada com este conjunto
            if (i+1) % 100 == 0:
                # ao não passar a operação de minimização, a rede não é treinada
                acc, stats = sess.run(
                    [accuracy, merged], 
                    feed_dict={X: test_inputs, y: test_labels}
                )
                # escrever as estatísticas de teste para o passo i
                test_writer.add_summary(
                    summary=stats, 
                    global_step=i
                )         
                print("Step {}: Test Accuracy= {}".format(i, acc))    

Step 99: Test Accuracy= 0.5850000000000001
Step 199: Test Accuracy= 0.61
Step 299: Test Accuracy= 0.62
Step 399: Test Accuracy= 0.635
Step 499: Test Accuracy= 0.785
Step 599: Test Accuracy= 0.8350000000000001
Step 699: Test Accuracy= 0.8350000000000001
Step 799: Test Accuracy= 0.8500000000000001
Step 899: Test Accuracy= 0.8450000000000001
Step 999: Test Accuracy= 0.855


## Visualização do treino

Corremos o Tensorboard com o seguinte commando, no directório onde foi corrido o script:

```
tensorboard --logdir=./logs --reload_interval=1
```

O primeiro argumento indica o directório onde foram escritas os sumários e o segundo o período de tempo de atualização dos dados visualizados.

Abrindo o browser na porta indicada (que por defeito é a 6006) vemos um ecrã inicial:

<img src="images/tensorboard_home.JPG" />

_post_ é o único namescope com operações summary do tipo escalar neste exemplo. Clicando na tab, podemos ver os gráficos produzidos

<img src="images/tensorboard_plots.JPG" />

Na secção da esquerda encontram-se os ficheiros que foram escritos e permite escolher quais são visualizados. Neste caso é criado um para treino, validação e teste para cada *run* com o *timestamp* em que foram corridos como nome.

As tabs *Distributions* e *Histograms* permitem visualizar as distribuições das variáveis (pesos e biases de cada camada) em função do step.

<img src="images/tensorboard_distributions.JPG" />
<img src="images/tensorboard_histograms.JPG" />

Finalmente, a tab *Graphs* permite visualizar o grafo de computação construído:

<img src="images/tensorboard_graphs.JPG" />

Os quadrados visíveis inicialmente representam os namescopes utilizados. Clicando sobre o "+", é possível ver as operações que os compõem.

<img src="images/tensorboard_layer.JPG" />


## Visualização do dataset

Podemos usar o *Embedding Visualizer* para projetar dados com muitas dimensões para 2 ou 3 de modo a visualizá-los.
Neste exemplo visualizamos o dataset de pacientes baseado nos seus atributos

In [9]:
tf.reset_default_graph()
EMBED_DIR = os.path.join(LOG_DIR, 'embeddings')

path_for_metadata =  os.path.join(EMBED_DIR, 'metadata.tsv')

# escrever metadados num ficheiro .tsv (tab separated values)
# para ponto de dados, escrevemos os atributos com que pretendemos que eles sejam etiquetados
with open(path_for_metadata,'w') as f:
    sep = "\t"
    f.write("Index")
    for field in data.dtype.names:
        if field[:5] == "class":
            continue
        f.write(sep + field)
    f.write(sep + "Labels\n")
    for index in range(len(inputs)):
        f.write("{}".format(index))
        for field_i, field in enumerate(data.dtype.names):
            if field[:5] == "class":
                continue
            # Tensorboard ainda não permite definir as cores dos pontos
            # usamos aqui um valor aleatório para assegurar que é usado um gradiente de cores
            f.write("{}{}".format(sep, data[field][index] + random.uniform(0, 0.0001)))
            
        f.write("{}{}\n".format(sep, 'ckd' if labels[index][0] == 1 else 'notckd'))

# tensor com os valores que irão afetar a projecção dos pontos no espaço
# neste caso queremos mostrá-los com base nos seus atributos
embedding_var = tf.Variable(inputs, name="attributes")
summary_writer = tf.summary.FileWriter(EMBED_DIR)

config = tf_projector.ProjectorConfig()

# podem ser adicionados mais que um tensor de embeddings
embedding = config.embeddings.add()
# nome identifica o tensor criado acima
embedding.tensor_name = embedding_var.name
# indicamos o caminho para o ficheiro onde se encontram os metadados
embedding.metadata_path = path_for_metadata

tf_projector.visualize_embeddings(summary_writer, config)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, os.path.join(EMBED_DIR, "model.ckpt"))
sess.close()

UnknownError: Failed to rename: .\logs\embeddings\model.ckpt.index.tempstate1701704182032771794 to: .\logs\embeddings\model.ckpt.index : Access is denied.
; Input/output error
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_DOUBLE], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, attributes/_1)]]

Caused by op 'save/SaveV2', defined at:
  File "c:\users\ricardo\appdata\local\programs\python\python35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\ricardo\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\ricardo\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\ricardo\appdata\local\programs\python\python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  Fi

Iniciando o TensorBoard com o comando:

```
tensorboard --logdir=./logs/embeddings --reload_interval=1
```

E clicando na tab *Embeddings*:

<img src="images/tensorboard_embedding_home.JPG"/>

Por defeito, os dados são mostrados com o algoritmo PCA, que escolhe os três eixos que maximizam a variância entre os pontos.

Clicando no dropdown "Color by" podemos seleccionar os atributos com que os pontos serão coloridos. Seleccionando "Label", os pontos de classes diferentes são coloridos de forma diferente.

<img src="images/tensorboard_embedding.JPG"/>

Os pontos podem ser seleccionados, mostrando o seu índice assim como outros pontos vizinhos. 

<img src="images/tensorboard_selection.JPG"/>

Conseguimos ver imediatamente uma distinção entre indivíduos com e sem a doença.

Podem também ser visualizados com o algoritmo t-SNE, com parâmetros modificáveis. Este algoritmo tenta agrupar espacialmente pontos com características semelhantes.

<img src="images/tensorboard_tsne.JPG"/>

Conseguimos ver que existem dois clusters principais de indivíduos sem a doença, enquanto que a maioria dos que têm a doença se encontram noutro. Isto explica a facilidade com que a rede neuronal consegue diagnosticar os pacientes.